In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime
import random

# Setup WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode (without GUI)
options.add_argument(f'user-agent={random.choice(["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36","Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0","Mozilla/5.0 (Macintosh; Intel Mac OS X 11.5; rv:90.0) Gecko/20100101 Firefox/90.0"])}')

# Function to create and return the WebDriver instance
def create_driver():
    return webdriver.Chrome(options=options)

def get_forex_factory_calendar():
    url = 'https://www.forexfactory.com/calendar'
    driver = create_driver()  # Initialize the WebDriver
    driver.get(url)
    
    # Locate the calendar table by its class name
    calendar_table = driver.find_element(By.CLASS_NAME, "calendar__table")
    calendar_data = []

    # Loop through each row and extract data
    for row in calendar_table.find_elements(By.TAG_NAME, "tr"):
        row_data = [td.text for td in row.find_elements(By.TAG_NAME, "td")]
        if row_data:
            calendar_data.append(row_data)
    
    driver.quit()  # Close the WebDriver
    return calendar_data

# Get the Forex Factory Calendar data
calendar_data = get_forex_factory_calendar()

# Filter out empty values from the data
filtered_data = [[i for i in entry if i != ''] for entry in calendar_data]

# Define the column names for the DataFrame
columns = ["Time", "Currency", "Impact", "Event", "Actual", "Forecast", "Previous"]

# Create a DataFrame from the filtered data
df = pd.DataFrame(filtered_data, columns=columns)
df = df.dropna(how='all')  # Drop rows where all elements are NaN or empty

# Highlight the current day's events
current_date = datetime.now().strftime("%b %d %Y")
df['Highlight'] = df['Time'].apply(lambda x: 'highlight' if x and current_date in x else '')

# Apply a yellow background to highlighted rows
df_highlighted = df.style.apply(lambda x: ['background-color: yellow' if x['Highlight'] == 'highlight' else '' for _ in x], axis=1)

# Display the styled DataFrame
df_highlighted

,Time,Currency,Impact,Event,Actual,Forecast,Previous,Highlight
2,Sun Sep 15,6:30pm,NZD,BusinessNZ Services Index,45.5,45.2,None,
3,All Day,JPY,Bank Holiday,None,None,None,None,
4,7:01pm,GBP,Rightmove HPI m/m,0.8%,-1.5%,None,None,
5,All Day,CNY,Bank Holiday,None,None,None,None,
7,Mon Sep 16,2:30am,CHF,PPI m/m,0.2%,0.1%,0.0%,
8,5:00am,EUR,Italian Trade Balance,6.74B,4.45B,5.15B,None,
9,EUR,Trade Balance,15.5B,14.9B,17.0B,None,None,
10,8:30am,CAD,Manufacturing Sales m/m,1.4%,0.7%,-1.7%,None,
11,USD,Empire State Manufacturing Index,11.5,-4.1,-4.7,None,None,
12,All Day,CNY,Bank Holiday,None,None,None,None,
